In [12]:
import pandas as pd
import random
import os

In [13]:
# Directory containing subdirectories for each housing
# Each subdirectory contains a csv file 
data_path = "../../data/housing_data/"


In [60]:
def extract_housing_info_from_name(housing):
    """
    Extracts the housing information from the csv file
    The info is as follows:
        - The first upper case letter is the type of housing (A : appartement, M : maison)
        - The numbers after the letter represent the surface of the housing
        - The number after the dash represents the number of people living in the housing
    """
    try:
        info = housing.split(".")[0].split("_")[2]
        # print(f"Housing {info}")
        h_type = info[0]
        info = info.replace(h_type, "")
        surface, nb_people, h_ref = info.split("-")
    except:
        h_ref = housing.split(" ")[1].replace(".csv", "").strip("()")
        info = housing.split(" ")[0].split("_")[2]
        h_type = info[0]
        # print(f"Ref: {h_ref}")
        # print(f"Type: {h_type}")
        info = info.replace(h_type, "")
        # print(f"Info: {info}")
        surface, nb_people = info.split("-")
        # print(f"Surface: {surface}")
        # print(f"Nb people: {nb_people}")
    return h_type, surface, nb_people, h_ref

In [61]:
extract_housing_info_from_name("data_maison_A110-2-4.csv")

('A', '110', '2', '4')

In [62]:
list_housings = os.listdir(data_path) # list of all the housings
len(list_housings)

26

In [63]:
print(list_housings)

['data A110-5', 'data M170-6', 'data M120-5', 'data A150-6', 'data M135-3', 'data M150-4', 'data M100-3', 'data A130-4', 'data M140-5', 'data M250-5', 'data M80-2', 'data A50-3', 'data M90-4', 'data M65-3', 'data A100-3', 'data A120-4', 'data A30-2', 'data M200-6', 'data M85-3', 'data M160-5', 'data M50-2', 'data A25-1', 'data M110-4', 'data M180-5', 'data A15-1', 'data A50-2']


In [66]:
df = pd.DataFrame()



for housing in list_housings:
    houses = os.listdir(data_path + housing)
    random_indexes = [random.randint(0, len(houses)) for i in range(10)]
    # print(random_indexes)
    random_houses = []
    for i in random_indexes:
        random_houses.append(houses[i])
    # print(random_houses)
    for house in random_houses:
        
        h_type, h_surface, nb_people, h_ref = extract_housing_info_from_name(house)
        print(data_path + housing + "/" + house + " :   " + h_type + " " + h_surface + " " + nb_people + " " + h_ref)
        
        temp = pd.read_csv(data_path + housing + "/" + house, header=[1],sep=',')
        #rename the columns Unnamed: 0 to timestamp
        temp.rename(columns={'Unnamed: 0':'date'}, inplace=True)
        #drop the column Unnamed: 1 (consommation totale)
        temp.drop('Unnamed: 1', axis=1, inplace=True)

        #Transpose
        temp = temp.T

        #make the first row as header
        temp['tmp_columns'] = temp.index
        temp = temp.reset_index(drop=True)
        temp.columns = temp.iloc[0]
        temp = temp.drop(temp.index[0])
        #make the column original_columns in first column
        cols = temp.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        temp = temp[cols]
        #rename the column date to timestamp
        temp.rename(columns={'date':'timestamp'}, inplace=True)

        temp = pd.melt(temp, id_vars=['timestamp'], var_name='date', value_name='consommation')

        temp["day"] = temp["date"].apply(lambda x: x.split("/")[1])
        temp["month"] = temp["date"].apply(lambda x: x.split("/")[0])
        temp["h_type"] = h_type
        temp["h_surface"] = h_surface
        temp["nb_people"] = nb_people
        temp["h_ref"] = h_ref
        temp.drop("date", axis=1, inplace=True)
        
        df = pd.concat([df, temp], axis=0)
    
df.reset_index(drop=True, inplace=True) 
df

../../data/housing_data/data A110-5/data_maison_A110-5-231.csv :   A 110 5 231
../../data/housing_data/data A110-5/data_maison_A110-5-194.csv :   A 110 5 194
../../data/housing_data/data A110-5/data_maison_A110-5-85.csv :   A 110 5 85
../../data/housing_data/data A110-5/data_maison_A110-5-300.csv :   A 110 5 300
../../data/housing_data/data A110-5/data_maison_A110-5-228.csv :   A 110 5 228
../../data/housing_data/data A110-5/data_maison_A110-5-104.csv :   A 110 5 104
../../data/housing_data/data A110-5/data_maison_A110-5-320.csv :   A 110 5 320
../../data/housing_data/data A110-5/data_maison_A110-5-228.csv :   A 110 5 228
../../data/housing_data/data A110-5/data_maison_A110-5-36.csv :   A 110 5 36
../../data/housing_data/data A110-5/data_maison_A110-5-172.csv :   A 110 5 172
../../data/housing_data/data M170-6/data_maison_M170-6-210.csv :   M 170 6 210
../../data/housing_data/data M170-6/data_maison_M170-6-216.csv :   M 170 6 216
../../data/housing_data/data M170-6/data_maison_M170-6-1

,timestamp,consommation,day,month,h_type,h_surface,nb_people,h_ref
0,0:00,0.6787,14,12,A,110,5,231
1,0:30,0.6416,14,12,A,110,5,231
2,1:00,0.8673,14,12,A,110,5,231
3,1:30,1.531,14,12,A,110,5,231
4,2:00,1.3221,14,12,A,110,5,231
...,...,...,...,...,...,...,...,...
13465915,21:30,0.26,1,1,A,50,2,163
13465916,22:00,0.3149,1,1,A,50,2,163
13465917,22:30,0.3799,1,1,A,50,2,163
13465918,23:00,0.3744,1,1,A,50,2,163


In [69]:
# calculate df memory usage in GB
df.memory_usage(deep=False).sum() / 1024**3

0.8026314973831177

In [9]:
df.to_csv(f"../../data/dataframes/df.csv", index=False)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13465920 entries, 0 to 13465919
Data columns (total 8 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   timestamp     object
 1   consommation  object
 2   day           object
 3   month         object
 4   h_type        object
 5   h_surface     object
 6   nb_people     object
 7   h_ref         object
dtypes: object(8)
memory usage: 821.9+ MB


In [70]:
df["consommation"] = df["consommation"].astype(float)
df["day"]       = df["day"].astype(int)
df["month"]     = df["month"].astype(int)
df["h_surface"] = df["h_surface"].astype(int)
df["nb_people"] = df["nb_people"].astype(int)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13465920 entries, 0 to 13465919
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   timestamp     object 
 1   consommation  float64
 2   day           int64  
 3   month         int64  
 4   h_type        object 
 5   h_surface     int64  
 6   nb_people     int64  
 7   h_ref         object 
dtypes: float64(1), int64(4), object(3)
memory usage: 821.9+ MB


In [14]:
df.memory_usage(deep=True).sum() / 1024**3

2.0013530999422073

In [73]:
type(df.h_ref[0])

str

In [74]:
df.to_csv("../../data/dataframes/df.csv", index = False)